### Init

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langchain_gigachat import GigaChat
from langchain_gigachat.chat_models import GigaChat
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langgraph_supervisor import create_supervisor
from pydantic import BaseModel
from rich import print as print


from blanks.prompts import (
	critique_prompt,
	extract_prompt,
	prompt_shell_agent,
	research_agent_prompt,
	supervisor_planing_prompt,
	defender_prompt_short
)
from func.methods_1 import (
	get_pentest_artifact,
	get_shell_output,
	# planning,
	tavily_search_tool,
)
from func.plan import PentestArtifacts

load_dotenv(find_dotenv())
log_file = os.environ.get('LOG_FILE', "/logs/report_task_1.md")
key = os.environ.get('GIGACHAT_API_KEY')
scope = os.environ.get('GIGACHAT_SCOPE')
ssl = os.environ.get('GIGACHAT_VERIFY_SSL_CERTS')


TAVILY_API_KEY = os.environ.get('TAVILY_API_KEY')


model_gc = GigaChat(credentials=key,
				model='GigaChat-2-Max',
				scope='GIGACHAT_API_CORP',
				temperature = 0,
				verify_ssl_certs = False,
				profanity_check=False
				)

model_ai = ChatOllama(model='gpt-oss:latest')
model = model_gc


In [3]:

# hide-cell
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
	pretty_message = message.pretty_repr(html=True)
	if not indent:
		print(pretty_message)
		return

	indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
	print(indented)


def pretty_print_messages(update, last_message=False):
	is_subgraph = False
	if isinstance(update, tuple):
		ns, update = update
		# skip parent graph updates in the printouts
		if len(ns) == 0:
			return

		graph_id = ns[-1].split(":")[0]
		print(f"Update from subgraph {graph_id}:")
		print("\n")
		is_subgraph = True

	for node_name, node_update in update.items():
		update_label = f"Update from node {node_name}:"
		if is_subgraph:
			update_label = "\t" + update_label

		print(update_label)
		print("\n")

		messages = convert_to_messages(node_update["messages"])
		if last_message:
			messages = messages[-1:]

		for m in messages:
			pretty_print_message(m, indent=is_subgraph)
		print("\n")

In [ ]:

ip = "10.62.34.10"
ip_port = "127.0.0.1:22222"

task2= f"""На объекте воздействия {ip} открыты дополнительные порты, которые не
требуются для функционирования контрольных сервисов. Необходимо закрыть
эти доступы не нарушив работоспособность контрольных сервисов.
На объекте воздействия настроен набор контрольных сервисов, которые должны сохранить свою работоспособность при решении
задачи:
a. NGINX, открывается рабочий сервис: 80 порт (WEB-страница).
b. Taskstate, рабочий порт 700
c. Cargo, рабочий порт 112
d.Zabbix, рабочие порты 10050, 10051."""
user_message = (f"Подключись к {ip}, пример команды для подключения  sshpass -p qwerty123 ssh admin@10.62.1.10 'echo qwerty123 | sudo -S iptables -F && echo qwerty123 | sudo -S iptables -P INPUT DROP && echo qwerty123 | sudo -S iptables -A INPUT -p tcp --dport 80 -j ACCEPT && echo qwerty123 | sudo -S iptables -A INPUT -p tcp --dport 700 -j ACCEPT && echo qwerty123 | sudo -S iptables -A INPUT -p tcp --dport 112 -j ACCEPT'"
    'Используй iptables для закрытия портов, которые не являются контрольными сервисами. ВАЖНО: Не закрывай 22 порт от ssh!'
                
				
				)



In [5]:
from typing import cast
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain_core.embeddings import Embeddings
import numpy as np
from langchain_gigachat.embeddings import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(credentials=key, 
								
								scope='GIGACHAT_API_CORP',
								verify_ssl_certs=False)


checkpointer = InMemorySaver()
store = InMemoryStore(index={
				"dims": 1024,
				"embed": cast(Embeddings ,embeddings),
				
			})
config = {"configurable": {"thread_id": "abc105"}}


namespace = ('hachaton_memory')

memory_tools = [
	create_manage_memory_tool(namespace),
	create_search_memory_tool(namespace),
]


### Init agents

In [ ]:
from operator import add
from pathlib import Path
from typing import Optional
from loguru import logger
from pydantic import Field
from typing_extensions import Annotated
from langgraph.graph.state import StateGraph
from langchain_core.messages import AnyMessage



from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
import pickle
from func.methods_1 import extract_pentest_artifacts
console_log = os.environ.get('CONSOLE_LOG_FILE', '/logs/my_console.md')
report_file = str(os.environ.get('LOG_FILE', 'report_task_1.md'))





def init_pentest_artifact() -> PentestArtifacts:
	"""Проверить хранилище данных о пентесте для получения информации.
	Args:
	task: str - текущая задача
	"""
	# global pentest_store
	pentest_store = PentestArtifacts()
	
	if os.path.exists('pentest_dump.pkl'):
		with open('pentest_dump.pkl', 'rb') as file:
			pentest_store = pickle.load(file)
			logger.debug("load from dump: " + str(pentest_store.model_dump()))
	else:
		pentest_store = PentestArtifacts()
		logger.debug("created new pentest_store")
		return pentest_store
	
	with open(report_file, "r") as file, open(console_log, "r") as file1, open("pentest_dump.pkl","wb") as dump:
		text = file.readlines()
		res = extract_pentest_artifacts('\n'.join(text))
		logger.debug(res)
		pentest_store.increment(res)

		text1 = file1.readlines()
		if len(text1) < 51:

			res = extract_pentest_artifacts('\n'.join(text1))
			pentest_store.increment(res)	
			logger.debug(res)
		else:
			for i in range(len(text1) // 10):
				# print(text[i*100:(i+1)*100])
				res = extract_pentest_artifacts('\n'.join(text1[i*50:(i+1)*50]))
				logger.debug(res)
				pentest_store.increment(res)	
		
		pickle.dump(pentest_store, dump)
		logger.debug(f"read from file {report_file}: " + str(pentest_store.model_dump()))
		logger.debug(f"read from file {console_log}: " + str(pentest_store.model_dump()))

	return pentest_store

In [7]:
from operator import add
from pathlib import Path
from typing import Optional
from loguru import logger
from pydantic import Field
from typing_extensions import Annotated
from langgraph.graph.state import StateGraph
from langchain_core.messages import AnyMessage



from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
import pickle
from func.methods_1 import extract_pentest_artifacts
console_log = os.environ.get('CONSOLE_LOG_FILE', '/logs/my_console.md')
report_file = str(os.environ.get('LOG_FILE', 'report_task_1.md'))



class PentestState(BaseModel):
	messages: Annotated[list[AnyMessage], add_messages]
	pentest_artifacts: Optional[Annotated[PentestArtifacts, PentestArtifacts.increment]] = Field(default_factory=PentestArtifacts)
	remaining_steps: int = 250

def prehook_pentest_artifact(
	state: PentestState,
) -> PentestState:
	"""
	Проверить/создать хранилище артефактов пентеста, обработать лог‑файл
	и вернуть обновлённый объект состояния.

	Parameters
	----------
	state : PentestState
		Текущее состояние (может быть пустым – оно будет перезаписано).
	log_file : Path
		Путь к лог‑файлу, который нужно обработать.
	task : str
		Текущая задача, по которой будет происходить извлечение.
	dump_path : Path, optional
		Путь к файлу дампа артефактов (по умолчанию `pentest_dump.pkl`).
	"""
	# 1️⃣  Загружаем существующий дамп или создаём новый
	dump_path: Path = Path("pentest_dump.pkl")
	if dump_path.is_file():
		with dump_path.open("rb") as f:
			pentest_store: PentestArtifacts = pickle.load(f)
	else:
		pentest_store = PentestArtifacts()

	# 2️⃣  Чтение лог‑файла по кускам по 100 строк
	log_file = os.environ.get('LOG_FILE', "/logs/report_task_1.md")
	log_file: Path = Path(log_file)
	if not log_file.is_file():
		raise FileNotFoundError(f"Log file not found: {log_file}")

	with log_file.open("r", encoding="utf-8") as f:
		chunk_lines: list[str] = []
		for line_no, line in enumerate(f, start=1):
			chunk_lines.append(line.rstrip("\n"))
			if line_no % 100 == 0:
				chunk = "\n".join(chunk_lines)
				res = extract_pentest_artifacts(chunk, task)
				pentest_store.increment(res)
				chunk_lines.clear()

		# обработка последнего неполного куска
		if chunk_lines:
			chunk = "\n".join(chunk_lines)
			res = extract_pentest_artifacts(chunk, task)
			pentest_store.increment(res)

	# 3️⃣  Сохраняем обновлённый дамп
	with dump_path.open("wb") as f:
		pickle.dump(pentest_store, f)

	# 4️⃣  Возвращаем новое состояние
	return PentestState(
		messages=state.messages,              # <-- передаем существующие сообщения
		pentest_artifacts=pentest_store,      # <-- ваш новый артефакт
		remaining_steps=state.remaining_steps # <-- оставляем прежнее число шагов
	)


def init_pentest_artifact() -> PentestArtifacts:
	"""Проверить хранилище данных о пентесте для получения информации.
	Args:
	task: str - текущая задача
	"""
	# global pentest_store
	pentest_store = PentestArtifacts()
	
	if os.path.exists('pentest_dump.pkl'):
		with open('pentest_dump.pkl', 'rb') as file:
			pentest_store = pickle.load(file)
			logger.debug("load from dump: " + str(pentest_store.model_dump()))
			return pentest_store
	else:
		pentest_store = PentestArtifacts()
		logger.debug("created new pentest_store")
		
	
	with open(report_file, "r") as file, open(console_log, "r") as file1, open("pentest_dump.pkl","wb") as dump:
		text = file.readlines()
		res = extract_pentest_artifacts('\n'.join(text))
		logger.debug(res)
		pentest_store.increment(res)

		text1 = file1.readlines()
		if len(text1) < 51:

			res = extract_pentest_artifacts('\n'.join(text1))
			pentest_store.increment(res)	
			logger.debug(res)
		else:
			for i in range(len(text1) // 10):
				# print(text[i*100:(i+1)*100])
				res = extract_pentest_artifacts('\n'.join(text1[i*50:(i+1)*50]))
				logger.debug(res)
				pentest_store.increment(res)	
		
		pickle.dump(pentest_store, dump)
		logger.debug(f"read from file {report_file}: " + str(pentest_store.model_dump()))
		logger.debug(f"read from file {console_log}: " + str(pentest_store.model_dump()))

	return pentest_store

In [8]:

model = model_gc

pentest_store = init_pentest_artifact()


shell_agent = create_react_agent(
	model=model,
	tools=[get_shell_output],
	# state_schema=PentestState,
	
	name="shell_comand_expert",
	prompt=prompt_shell_agent
)

web_search_agent = create_react_agent(
	model=model,
	tools=[tavily_search_tool],
	name="web_search_expert",
	prompt=research_agent_prompt
)


defender_agent = create_react_agent(
	model=model,
	tools=[get_shell_output, ], # *file_system_tools
	name="defender_expert",
	prompt=defender_prompt_short
)


critique_agent = create_react_agent(
	model=model,
	tools=[get_pentest_artifact],
	name="critique_expert",
	prompt= critique_prompt.format(task = task2, pentest_store = pentest_store)
)






workflow_plan = create_supervisor(
	agents = [shell_agent, critique_agent, web_search_agent, defender_agent], #fallback_agent], #
	model=model,
	# pre_model_hook=prehook_pentest_artifact,
	# state_schema=PentestState,
	tools=[get_pentest_artifact, get_shell_output,  ], # *file_system_tools,

	output_mode="full_history",
	prompt=supervisor_planing_prompt,
	
	
)
planing_attacker = workflow_plan.compile( checkpointer=checkpointer, store=store)

2025-08-30 20:43:57.906 | DEBUG    | __main__:init_pentest_artifact:97 - load from dump: {'credentials': [{'type': 'password', 'login': 'admin', 'password': 'protect', 'context': 'ssh'}, {'type': 'password', 'login': 'uftp', 'password': 'clock', 'context': 'ftp'}, {'type': 'password', 'login': 'admin', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': '123456', 'context': ''}, {'type': 'password', 'login': 'user', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'user', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'user', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'user', 'password': '123456', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': '123456

### Решение


In [9]:
planing_attackerconfig = {"configurable": {"thread_id": "114"}, "recursion_limit": 350 }
message = {
	"messages": [
		{
			"role": "user",
			"content": (f"Вот задание: {task2.format(ip)} "
				"Составь план решения задачи, вот примерный план:"
			
				"1. Подключись к PC {ip} через sshpass. Пароль от ssh возьми из хранилища данных. Обязательно подключись, без подключения задача не решена."
				"2. Определи сервисы, порты к которым можно закрыть."
				"2. Используй iptables, чтоб закрыть сервисы, которые не являются контрольными сервисами."
				"""3. Пример подключения: ssshpass -p <password>  ssh admin@10.62.1.10 'echo <password> | sudo -S iptables -F && echo <password>  | sudo -S iptables -P INPUT DROP && echo <password>  | sudo -S iptables -A INPUT -p tcp --dport 80 -j ACCEPT && echo <password>  | sudo -S iptables -A INPUT -p tcp --dport 700 -j ACCEPT && echo <password>  | sudo -S iptables -A INPUT -p tcp --dport 112 -j ACCEPT'""")


		}
	]
}



for chunk in planing_attacker.stream(message, planing_attackerconfig):
	pretty_print_messages(chunk, last_message=True)

2025-08-30 20:43:59.565 | DEBUG    | func.methods_1:get_pentest_artifact:100 - load from dump: {'credentials': [{'type': 'password', 'login': 'admin', 'password': 'protect', 'context': 'ssh'}, {'type': 'password', 'login': 'uftp', 'password': 'clock', 'context': 'ftp'}, {'type': 'password', 'login': 'admin', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': '123456', 'context': ''}, {'type': 'password', 'login': 'user', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'user', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'user', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'user', 'password': '123456', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': '

Update from node supervisor:

=================================[1m Tool Message [0m=================================
Name: transfer_to_critique_expert

Successfully transferred to critique_expert

Update from node critique_expert:

=================================[1m Tool Message [0m=================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor

Update from node supervisor:

==================================[1m Ai Message [0m==================================
Name: supervisor

Финальное решение прошло проверку у эксперта-критика. Задача считается выполненной успешно.